In [1]:
import import_ipynb
import pandas as pd
import numpy as np
import torch
from datetime import datetime
from datetime import datetime as dt
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler
import pickle

In [2]:
from feeds import BackFeed,DataFeed
from featfuncs import feat_aug,add_addl_features_feed,add_ta_features_feed,add_sym_feature_feed
from featfuncs import add_global_indices_feed

importing Jupyter notebook from feeds.ipynb
importing Jupyter notebook from featfuncs.ipynb
importing Jupyter notebook from india_calendar.ipynb


In [3]:
from feed_env import Episode

importing Jupyter notebook from feed_env.ipynb


In [4]:
from ruleagents import AdaMomCMOAgent, GapBetAgent

importing Jupyter notebook from ruleagents.ipynb
importing Jupyter notebook from aiagentbase.ipynb


In [5]:
from rulestrats import AdaMomCMOADF, GapBet

importing Jupyter notebook from rulestrats.ipynb


In [18]:
from backtest import Backtest

importing Jupyter notebook from backtest.ipynb


In [14]:
import aspectlib

In [6]:
def do_nothing(dfD):
    empty={t:0 for t in dfD}
    return empty,empty,empty
def always_buy(dfD):
    buy={t:1 for t in dfD}
    empty={t:0 for t in dfD}
    return buy,empty,empty
def always_sell(dfD):
    sell={t:-1 for t in dfD}
    empty={t:0 for t in dfD}
    return sell,empty,empty

In [7]:
def get_alt_data():
    global feed
    aD={'gdata':feed.gdata}
    return aD

Host Service via Anvil

TBD: enable remote calling for agent strategies - add reward function here and call in tradeserver

In [8]:
import anvil.server

In [ ]:
# DeepTrader:
anvil.server.connect("STKYCIXUMGFV2CURGMCMOFUV-MBK5ZXFC4MCKYKOE")

In [ ]:
momStrat=MomStrat()

In [ ]:
momStrat=AdaMomCMOADF()

In [ ]:
@anvil.server.callable
def check_entry_batch_mom2(dfD):
    global momStrat
    dfDloc={}
    for t in dfD.keys():
        dfDloc[t]=pd.DataFrame(dfD[t])
    return MomStrat.Check(momStrat,dfDloc)

In [ ]:
@anvil.server.callable
def check_exit_batch_mom2(dfD,posf):
    global momStrat
    posloc=pd.DataFrame(posf)
    dfDloc={}
    for t in dfD.keys():
        dfDloc[t]=pd.DataFrame(dfD[t])
    return MomStrat.Exit(momStrat,dfDloc,posloc)

Strategy Development: ADF Momentum

In [ ]:
from backtest import Backtest
from feeds import BackFeed,DataFeed
from validation import Validate

In [ ]:
data=pd.read_csv('./capvol100.csv')

In [ ]:
tickers=list(data.iloc[0:10]['ticker'].values)

In [ ]:
feed=BackFeed(tickers=tickers,nd=3,nw=1,interval='5m')

In [ ]:
add_addl_features_feed(feed,tickers=feed.tickers)

In [ ]:
add_sym_feature_feed(feed,tickers=feed.tickers)

In [ ]:
add_global_indices_feed(feed)

In [9]:
import pickle

In [10]:
# with open('../../temp_data/btfeed.pickle','wb') as f: pickle.dump(feed,f)

In [11]:
with open('../../temp_data/btfeed.pickle','rb') as f: feed=pickle.load(f)

In [ ]:
gapagent=GapBetAgent(direction=-1)

In [ ]:
gapagent=GapBetAgent(direction=-1)
gapagent.set_alt_data(alt_data_func=get_alt_data)

In [ ]:
gapstrat=GapBet()
gapstrat.set_alt_data(alt_data_func=get_alt_data)

In [12]:
adamom=AdaMomCMOAgent()

In [ ]:
adastrat=AdaMomCMOADF()

In [ ]:
import importlib
import sys,os

In [ ]:
importlib.reload(sys.modules['feed_env'])
from feed_env import Episode

In [ ]:
importlib.reload(sys.modules['backtest'])
from backtest import Backtest

In [ ]:
importlib.reload(sys.modules['rulestrats'])
from rulestrats import AdaMomCMOADF,GapBet

In [ ]:
# def save_func(df): return(df.iloc[-1]['CMO_14'])

In [ ]:
feed.ncounter

In [15]:
@aspectlib.Aspect
def my_decorator(*args, **kwargs):
    # print("Got called with args: %s kwargs: %s" % (args, kwargs))
    # result = yield
    # print(" ... and the result is: %s" % (result,))
    state,rew,done,exit_type = yield
    args[0].policy.reward(rew)
    return state,rew,done,exit_type

In [16]:
aspectlib.weave(Episode, my_decorator, methods='env_step')

In [19]:
bt=Backtest(feed,tickers=feed.tickers,add_features=False,target=.5,stop=.5,txcost=0.001,
            loc_exit=True,scan=True,topk=3,deploy=True,save_dfs=False,
            save_func=None)

In [20]:
bt.run_all(tickers=feed.tickers,model=adamom,verbose=False)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [23]:
bt.returns

{'DMART.NS': -0.6671137133538629,
 'ADANITRANS.NS': -2.7225752010283686,
 'MARUTI.NS': -0.4031886495179515,
 'NESTLEIND.NS': -0.029027437530623038,
 'NTPC.NS': 0.09732813494630306}

In [ ]:
bt.results

In [ ]:
from datetime import datetime

In [ ]:
datetime.today()

In [ ]:
bt.returns

In [27]:
M=adamom.memory.sar_memory

In [29]:
[M[t]['reward'] for t in M if 'reward' in M[t]]

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 -0.7561692582645189,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 -0.4031886495179515,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 -0.6671137133538629,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 -1.9664059427638496,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 -0.029027437530623038,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.09732813494630306]

In [ ]:
M